In [ ]:
!pip install "numpy<2.0" opencv-python-headless ultralytics huggingface_hub


[notice] A new release of pip is available: 24.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
from ultralytics import YOLO
from huggingface_hub import hf_hub_download
import os
from collections import deque

In [3]:
# --- STEP 1: LOAD "LITTLE YOLO" ---
# We use 'yolov8n.pt' (n = nano). It is the smallest, fastest model.
# It automatically downloads from Hugging Face / Ultralytics if not found.
model = YOLO('yolov8n.pt')
print("Model loaded successfully.")

Model loaded successfully.


In [ ]:
def save_clean_chess_frames(video_path, output_folder="clean_frame", save_step = True):
    os.makedirs(output_folder, exist_ok=True)
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error: Cannot open video.")
        return

    # --- SETTINGS ---
    FRAME_SKIP = 15       # Look at 1 frame, skip 14
    CONFIDENCE = 0.125   # Be 12.5% sure it's a hand
    BUFFER_SIZE = 2       # Must see same state 2 times to switch
    
    # --- STATE VARIABLES ---
    frame_count = 0
    chunk_counter = 1     # We will number our chunks: 1, 2, 3...
    
    current_official_state = None # current state
    
    # store tuples: (frame_image, frame_number, state_string)
    result_buffer = deque(maxlen=BUFFER_SIZE)

    print(f"Processing... Saving ordered chunks to '/{output_folder}'")

    while True:
        ret, frame = cap.read()
        if not ret: break
        
        frame_count += 1
        if frame_count % FRAME_SKIP != 0: continue # Skip frames for speed

        # 1. DETECT
        results = model(frame, verbose=False, conf=CONFIDENCE, classes=[0]) 
        
        is_blocked = False
        for result in results:
            if len(result.boxes) > 0: 
                is_blocked = True
                break
        
        frame_state = "blocked" if is_blocked else "clean"
        
        # 2. ADD TO BUFFER
        # We save the frame data so we can go back and save it later if needed
        result_buffer.append((frame.copy(), frame_count, frame_state))
        
        # We need at least 3 frames in buffer to make a decision
        if len(result_buffer) < BUFFER_SIZE:
            continue

        # 3. CHECK FOR CONSENSUS
        # Extract just the states from the buffer: e.g., ['clean', 'clean', 'clean']
        recent_states = [x[2] for x in result_buffer]
        
        # Check if all 3 states in the buffer are identical
        if all(s == recent_states[0] for s in recent_states):
            detected_stable_state = recent_states[0]
            
            # --- SCENARIO A: FIRST CHUNK EVER ---
            if current_official_state is None:
                current_official_state = detected_stable_state
                
                # Save the START image (use the oldest one in buffer)
                img, num, state = result_buffer[0]
                filename = f"{output_folder}/{chunk_counter:02d}_frame{num}_START_{state}.jpg"
                cv2.imwrite(filename, img)
                print(f"[{chunk_counter:03d}] Started {state} at frame {num}")

            # --- SCENARIO B: STATE SWITCH (e.g. Clean -> Blocked) ---
            elif current_official_state != detected_stable_state:
                
                # 1. Close the OLD chunk
                # We use the OLDEST frame in buffer as the "End" of the previous state
                # because if the buffer is [Block, Block, Block], the Clean state ended right before this buffer.
                if(save_step):
                    img_end, num_end, _ = result_buffer[0] 
                    filename_end = f"{output_folder}/{chunk_counter:02d}_frame{num_end}_END_{current_official_state}.jpg"
                    cv2.imwrite(filename_end, img_end)
                
                # 2. Start the NEW chunk
                chunk_counter += 1 # Increment number (1 -> 2)
                current_official_state = detected_stable_state
                
                # Use the NEWEST frame in buffer as the "Start" of the new state
                img_start, num_start, state_start = result_buffer[-1]
                if state_start == "clean" or save_step:
                    filename_start = f"{output_folder}/{chunk_counter:02d}_frame{num_start}_START_{state_start}.jpg"
                    cv2.imwrite(filename_start, img_start)
                
                print(f"[{chunk_counter:03d}] Switched to {state_start} at frame {num_start}")

    # --- FINAL CLEANUP ---
    if len(result_buffer) > 0 and current_official_state is not None and save_step:
        img_last, num_last, _ = result_buffer[-1]
        filename_final = f"{output_folder}/{chunk_counter:02d}_frame{num_last}_END_{current_official_state}.jpg"
        cv2.imwrite(filename_final, img_last)

    cap.release()
    print("Done.")

In [27]:
# --- RUN IT ---
video_file = "test_videos/8_Move_student.mp4" 
save_clean_chess_frames(video_file,output_folder="8_Move_student",save_step=False)

Processing... Saving ordered chunks to '/8_Move_student'
[001] Started clean at frame 15
[002] Switched to blocked at frame 1140
[003] Switched to clean at frame 1380
[004] Switched to blocked at frame 1410
[005] Switched to clean at frame 1470
[006] Switched to blocked at frame 1545
[007] Switched to clean at frame 1635
[008] Switched to blocked at frame 1665
[009] Switched to clean at frame 2010
[010] Switched to blocked at frame 4500
[011] Switched to clean at frame 4815
[012] Switched to blocked at frame 7380
[013] Switched to clean at frame 7425
[014] Switched to blocked at frame 10305
[015] Switched to clean at frame 10365
[016] Switched to blocked at frame 10635
[017] Switched to clean at frame 10740
[018] Switched to blocked at frame 10770
[019] Switched to clean at frame 10815
[020] Switched to blocked at frame 11385
[021] Switched to clean at frame 11595
[022] Switched to blocked at frame 12045
[023] Switched to clean at frame 12225
[024] Switched to blocked at frame 13695
[0